In [105]:
from tmu.tsetlin_machine import TMCoalescedClassifier
import numpy as np 
import pickle
import torch
from tabulate import tabulate
import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from itertools import combinations
# from apriori_python import apriori
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import seaborn as sns
!export OMP_NUM_THREADS=10
n_clauses_per_class = 400
treshold = int(n_clauses_per_class * 0.8)
s_param = 11.40000000000001
# s_param = 5

"s henger sammen hvor ofte du ser pattern"
"lav s sier at man ser et pattern ofte, og kan glemme dette"
"belonning er s/(s-1)"
"random penalty er 1/s"

'random penalty er 1/s'

In [89]:
cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0, feature_negation=False)


In [3]:
game_amount = 1000000
# tsetlin_filename, self_can_bet_index= "/home/jaoi/master22/pet_for_sale/winning_games_db/{}_tsetlined_games.pkl".format(game_amount), 17
tsetlin_filename, self_can_bet_index = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_tsetlined_games.pkl".format(game_amount), 21

with open(tsetlin_filename, "rb") as fp:
        games = pickle.load(fp)

In [4]:
# preprossesing
def remove_duplicate_pairs(games):
    new_games = []
    for i,game in enumerate(games):
        int_arr = game[0].astype(int)
        complete_arr = np.append(int_arr, game[1])
        new_games.append(complete_arr)

    new_games = np.array(new_games)
    
    x = np.random.rand(new_games.shape[1])
    y = new_games.dot(x)
    unique, index = np.unique(y, return_index=True)
    unique_xy = new_games[index]
    
    unique_pairs = []
    for game in unique_xy:
        last, rest = game[-1], game[:-1]
        unique_pairs.append(np.array((rest.astype(np.float32), last), dtype=object))
    return np.array(unique_pairs)


def oversample(info_tup, desired_amount, source_games):
    bob = []
    for game in source_games:
        if game[1] == info_tup[0]:
            bob.append(game)
    indexes = np.random.randint(0, info_tup[1], desired_amount - info_tup[1])
    temp = []
    for ind in indexes:
        temp.append(bob[ind])
    return np.array(temp)


def preprocess_games(games):
    
    where_can = []
    for i, game in enumerate(games):
        if game[0][self_can_bet_index] == 1:
            where_can.append(i)
    indexes = np.array(where_can)
    print("past indexes")
    games_can_bid = games[indexes]

    unique_pairs = remove_duplicate_pairs(games_can_bid)
    print("past unique pairs")
    ysu = Counter()
    for game in unique_pairs:
        ysu[game[1]] += 1
    
    need_of_oversample = [tup for tup in ysu.most_common() if tup[1] < 2000]
    print("past unique need of oversample")
    


    unique_oversample = np.copy(unique_pairs)
    temp = []
    for overnd in need_of_oversample:
        temp.extend(oversample(overnd, 2000, unique_pairs))
    unique_oversample = np.concatenate((unique_oversample, np.array(temp)), axis=0)
    print("past unique oversample")
    
    uo = Counter()
    for game in unique_oversample:
        uo[game[1]] += 1
    print(uo)
    return unique_oversample

unique_oversample = preprocess_games(games[:int(games.shape[0]*0.2)])

past indexes
past unique pairs
past unique need of oversample
past unique oversample
Counter({0: 255068, 4: 153605, 6: 108546, 2: 83675, 3: 78002, 8: 67281, 5: 53634, 7: 38587, 10: 26941, 9: 24848, 11: 10307, 12: 7002, 13: 2629, 1: 2000, 14: 2000, 15: 2000, 16: 2000})


In [5]:
x =np.array([game[0] for game in unique_oversample[:, :1]])
y =np.array([game[0] for game in unique_oversample[:, 1:]])


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2)
# remove self bid feature, which is at index 21
x_train = np.delete(x_train, 21, 1)
x_test = np.delete(x_test, 21, 1)


In [7]:
# split out the different classes, and check if some of the classes are making it harder for the TM

In [8]:
# train_i = int(unique_oversample.shape[0] * 0.9)
# np.random.shuffle(unique_oversample)
# x_train =np.array([game[0] for game in unique_oversample[:train_i, :1]])
# x_test =np.array([game[0] for game in unique_oversample[train_i:, :1]])

# y_train =np.array([game[0] for game in unique_oversample[:train_i, 1:]])
# y_test =np.array([game[0] for game in unique_oversample[train_i:, 1:]])

In [130]:
epochs = 5
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    cuda_tm.fit(x_train, y_train)

    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_tm.predict(x_test) == y_test).mean())

Epoch 0 Accuracy tm: 62.699523485364196
Epoch 1 Accuracy tm: 59.11341048332199
Epoch 2 Accuracy tm: 61.366916269571135
Epoch 3 Accuracy tm: 60.24179714091219
Epoch 4 Accuracy tm: 59.92920353982301


In [8]:
cuda_e1_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0)
cuda_e2_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0)
cuda_e3_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0)

In [9]:
epochs = 1
for epoch in range(epochs):
    cuda_e1_tm.fit(x_train, y_train)
    cuda_e2_tm.fit(x_train, y_train)
    cuda_e3_tm.fit(x_train, y_train)
    
    

    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_e1_tm.predict(x_test) == y_test).mean())
    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_e2_tm.predict(x_test) == y_test).mean())    
    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_e3_tm.predict(x_test) == y_test).mean())    

Epoch 0 Accuracy tm: 79.9498978897209
Epoch 0 Accuracy tm: 79.96133424098026
Epoch 0 Accuracy tm: 79.75057862491491


In [8]:
def get_name_of_feature_from_3_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    reverse_2_bits = [1, 0]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr == 17:
        return "self can bid bit"
    elif feature_nr in [18,19,20,21,22]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 18])
    elif feature_nr in [23,24,25,26,27]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 23])
    elif feature_nr in [28,29,30,31]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 28])
    elif feature_nr in [32,33]:
        return "self next turn order {}".format(2**reverse_2_bits[feature_nr - 32])
    # for next player in line
    elif feature_nr in [34, 35, 36, 37, 38]:
        return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 34]))
    elif feature_nr in [39, 40, 41, 42]:
        return "next player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 39])
    elif feature_nr in [43, 44, 45, 46]:
        return "next player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 43])
    elif feature_nr in [47, 48, 49, 50]:
        return "next player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 47])
    elif feature_nr == 51:
        return "next can bid bit"
    elif feature_nr in [52, 53, 54, 55, 56]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 52])
    elif feature_nr in [57, 58, 59, 60, 61]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 57])
    elif feature_nr in [62, 63, 64, 65]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 62])
    elif feature_nr in [66, 67]:
        return "next player turn order {}".format(2**reverse_2_bits[feature_nr - 66])
    # for last player in line
    elif feature_nr in [68, 69, 70, 71, 72]:
        return "last player coins bit {}".format(2**(reverse_5_bits[feature_nr - 68]))
    elif feature_nr in [73, 74, 75, 76]:
        return "last player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 73])
    elif feature_nr in [77, 78, 79, 80]:
        return "last player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 77])
    elif feature_nr in [81, 82, 83, 84]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 81])
    elif feature_nr == 85:
        return "last player can bid"
    elif feature_nr in [86, 87, 88, 89, 90]:
        return "current next player bid {}".format(2**reverse_5_bits[feature_nr - 86])
    elif feature_nr in [91, 92, 93, 94, 95]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 91])
    elif feature_nr in [96, 97, 98, 99]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 96])
    elif feature_nr in [100, 101]:
        return "last player turn order {}".format(2**reverse_2_bits[feature_nr - 100])
    # remainder stats
    elif feature_nr in [102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]:
        return "{}. largest in stack".format(feature_nr - 101)
    elif feature_nr in [113, 114, 115, 116]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 113])
    elif feature_nr in [117, 118, 119, 120]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 117])
    elif feature_nr in [121, 122, 123, 124]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 121])
    elif feature_nr == 125:
        return "board_std"
    return "not implemented"

In [10]:
def get_name_of_feature_from_4_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    reverse_2_bits = [1, 0]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr in [17,18,19,20]:
        return "own nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 17])
    elif feature_nr in [21,22,23,24,25]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 21])
    elif feature_nr in [26,27,28,29,30]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 26])
    elif feature_nr in [31,32,33,34]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 31])
    elif feature_nr in [35,36]:
        return "self next turn order {}".format(2**reverse_2_bits[feature_nr - 35])
    # for next player in line
    elif feature_nr in [37,38,39,40,41]:
        return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 37]))
    elif feature_nr in [42,43,44,45]:
        return "next player nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 42])
    elif feature_nr in [46,47,48,49]:
        return "next player nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 46])
    elif feature_nr in [50,51,52,53]:
        return "next player nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 50])
    elif feature_nr in [54,55,56,57]:
        return "next player nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 54])
    elif feature_nr == 58:
        return "next can bid bit"
    elif feature_nr in [59, 60, 61, 62, 63]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 59])
    elif feature_nr in [64, 65, 66, 67, 68]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 64])
    elif feature_nr in [69, 70, 71, 72]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 69])
    elif feature_nr in [73, 74]:
        return "next player turn order {}".format(2**reverse_2_bits[feature_nr - 73])
    # for last player in line
    elif feature_nr in [75,76,77,78,79]:
        return "final player coins bit {}".format(2**(reverse_5_bits[feature_nr - 75]))
    elif feature_nr in [80,81,82,83]:
        return "final player nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 80])
    elif feature_nr in [84,85,86,87]:
        return "final player nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 84])
    elif feature_nr in [88,89,90,91]:
        return "final player nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 88])
    elif feature_nr in [92,93,94,95]:
        return "final player nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 92])
    elif feature_nr == 96:
        return "final player can bid"
    elif feature_nr in [97,98, 99, 100, 101]:
        return "current final player bid {}".format(2**reverse_5_bits[feature_nr - 97])
    elif feature_nr in [102,103, 104, 105, 106]:
        return "resulting final player coins if fold {}".format(2**reverse_5_bits[feature_nr - 102])
    elif feature_nr in [107,108, 109, 110]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 107])
    elif feature_nr in [111, 112]:
        return "final player turn order {}".format(2**reverse_2_bits[feature_nr - 111])
    # remainder stats
    elif feature_nr in [113,114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]:
        return "{} stack bit on".format(feature_nr - 113)
    elif feature_nr in [128,129, 130, 131]:
        return "board cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 128])
    elif feature_nr in [132,133, 134, 135]:
        return "board cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 132])
    elif feature_nr in [136, 137, 138, 139]:
        return "board cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 136])
    elif feature_nr == 140:
        return "board_std"
    return "not implemented"
    # elif feature_nr == 21:
    #     return "self can bid bit"
    # elif feature_nr in [22,23,24,25,26]:
    #     return "current own bid {}".format(2**reverse_5_bits[feature_nr - 22])
    # elif feature_nr in [27,28,29,30,31]:
    #     return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 27])
    # elif feature_nr in [32,33,34,35]:
    #     return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 32])
    # elif feature_nr in [36,37]:
    #     return "self next turn order {}".format(2**reverse_2_bits[feature_nr - 36])
    # # for next player in line
    # elif feature_nr in [38,39,40,41,42]:
    #     return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 38]))
    # elif feature_nr in [43,44,45,46]:
    #     return "next player nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 43])
    # elif feature_nr in [47,48,49,50]:
    #     return "next player nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 47])
    # elif feature_nr in [51,52,53,54]:
    #     return "next player nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 51])
    # elif feature_nr in [55,56,57,58]:
    #     return "next player nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 55])
    # elif feature_nr == 59:
    #     return "next can bid bit"
    # elif feature_nr in [60, 61, 62, 63, 64]:
    #     return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 60])
    # elif feature_nr in [65, 66, 67, 68, 69]:
    #     return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 65])
    # elif feature_nr in [70, 71, 72, 73]:
    #     return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 70])
    # elif feature_nr in [74, 75]:
    #     return "next player turn order {}".format(2**reverse_2_bits[feature_nr - 74])
    # # for last player in line
    # elif feature_nr in [76,77,78,79,80]:
    #     return "final player coins bit {}".format(2**(reverse_5_bits[feature_nr - 76]))
    # elif feature_nr in [81,82,83,84]:
    #     return "final player nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 81])
    # elif feature_nr in [85,86,87,88]:
    #     return "final player nr 2 card bit {}".format(2**reverse_4_bits[feature_nr - 85])
    # elif feature_nr in [89,90,91,92]:
    #     return "final player nr 3 card bit {}".format(2**reverse_4_bits[feature_nr - 89])
    # elif feature_nr in [93,94,95,96]:
    #     return "final player nr 4 card bit {}".format(2**reverse_4_bits[feature_nr - 93])
    # elif feature_nr == 97:
    #     return "final player can bid"
    # elif feature_nr in [98, 99, 100, 101, 102]:
    #     return "current final player bid {}".format(2**reverse_5_bits[feature_nr - 98])
    # elif feature_nr in [103, 104, 105, 106, 107]:
    #     return "resulting final player coins if fold {}".format(2**reverse_5_bits[feature_nr - 103])
    # elif feature_nr in [108, 109, 110, 111]:
    #     return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 108])
    # elif feature_nr in [112, 113]:
    #     return "final player turn order {}".format(2**reverse_2_bits[feature_nr - 112])
    # # remainder stats
    # elif feature_nr in [114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]:
    #     return "{} stack bit on".format(feature_nr - 114)
    # elif feature_nr in [129, 130, 131, 132]:
    #     return "board cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 129])
    # elif feature_nr in [133, 134, 135, 136]:
    #     return "board cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 133])
    # elif feature_nr in [137, 138, 139, 140]:
    #     return "board cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 137])
    # elif feature_nr == 141:
    #     return "board_std"
    # return "not implemented"

In [122]:
def show_coalesced_clause_3r(tm, clause):
    number_of_features = 126
    print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_3_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
    return " ∧ ".join(l)

def show_coalesced_clause_4r(tm, clause):
    number_of_features = 142
    # print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_4_rounds(k-number_of_features))
    return " ∧ ".join(l)

def get_string_coalesced_clause_4r(tm, clause):
    number_of_features = 142
    # print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(get_name_of_feature_from_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_4_rounds(k-number_of_features))
    return l

def show_coalesced_weight(tm, the_class, weight):
    mw = tm.weight_banks[the_class].get_weights()
    return mw[weight]


In [92]:
def show_activated_clauses_location(tm, x):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1]
#     moves_and_scores = []
#     classes = len(tm.weight_banks)
#     for clas in range(classes):
#         mw = cuda_tm.weight_banks[clas].get_weights()
#         # print(np.argpartition(mw, -5)[-5:])
#         moves_and_scores.append((clas, np.dot(c_out, mw)))
#     top_moves = sorted(moves_and_scores, reverse=True, key=lambda x:x[1])

#     if top_moves[0][0] != tm.predict(x.reshape(1, -1))[0]:
#         print("OH OH <STINKY!> {} {}".format(top_moves[:5], tm.predict(x.reshape(1, -1))[0]))
#         return True
#     else:
#         False
    return clauses_out
show_activated_clauses_location(cuda_tm, x_test[0])

array([  1,  10,  20,  22,  30,  59,  62, 100, 116, 124, 140, 189, 205,
       213, 215, 232, 261, 277, 306, 316, 365, 397])

In [106]:
def tabulate_most_important_clauses_per_class(tm):
    classes = len(tm.weight_banks)
    for clas in range(classes):
        mw = tm.weight_banks[clas].get_weights()
        #largest for
        table = [["clauses for class {}". format(clas)]]
        for c in np.argpartition(mw, -5)[-5:]:
            table.append([show_coalesced_clause_4r(tm, c).strip().split("∧")])
            # print("{} weight, rule: {}".format(mw[c], show_coalesced_clause_4r(tm, c)))
        print(tabulate(table))
        print()
# tabulate_most_important_clauses_per_class(cuda_tm)

In [107]:
def show_most_important_clauses_per_class(tm):
    classes = len(tm.weight_banks)
    for clas in range(classes):
        mw = tm.weight_banks[clas].get_weights()
        #largest for
        print("clauses for class {}". format(clas))
        for c in np.sort(np.argpartition(mw, -5)[-5:]):
            print("{} weight, rule: {}".format(mw[c], show_coalesced_clause_4r(tm, c)))
        print()
        #largest against
        # print(np.argpartition(mw, 5)[:5])
# show_most_important_clauses_per_class(cuda_tm)

In [96]:
def prepare_tm_for_apriori(x, y, tm, sigma=0):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1].astype(int)
    mw = tm.weight_banks[y].get_weights()
    weight_to_plot = mw[clauses_out]
    clauses_out_r = clauses_out[abs(weight_to_plot) > sigma]
    return clauses_out_r

In [131]:
def single_tm_apriori(x, y, tm, sigma=-100000):
    pre_ap = prepare_tm_for_apriori(x, y, tm, sigma)
    check_clause = [show_coalesced_clause_4r(tm, clause).strip().split("∧") for i, clause in enumerate(pre_ap) if show_coalesced_weight(cuda_tm, y_test[20], clause) > 0]
    new_check = []
    for clause in check_clause:
        temp = []
        for feat in clause:
            temp.append(feat.strip())
        new_check.append(temp)
    te = TransactionEncoder()
    te_ary = te.fit(new_check).transform(new_check)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    return df

df = single_tm_apriori(x_test[20], y_test[20], cuda_tm)
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
pd.set_option('max_colwidth', 1000)
frequent_itemsets

,support,itemsets,length
0,0.250,(current next player bid 2 bit),1
1,0.125,(current next player bid 4 bit),1
2,0.125,(current own bid 4),1
3,0.125,(final player coins bit 16),1
4,0.250,(final player turn order 2),1
5,0.125,(next player turn order 1),1
6,0.500,(resulting final player coins if fold 16),1
7,0.125,(resulting next player coins if fold 8),1
8,0.125,(self next turn order 2),1
9,0.125,"(current own bid 4, current next player bid 2 bit)",2


In [128]:
def get_combinations(tm, x):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1].astype(int)
    feature_list_per_clause = [get_string_coalesced_clause_4r(cuda_tm, clause) for i, clause in enumerate(clauses_out)]
    comb_list = []
    for feat_list in feature_list_per_clause:
        for comb in combinations(feat_list, 2):
            comb_list.append(comb)
    c = Counter(comb_list)
    print(c.most_common())
    
get_combinations(cuda_tm, x_test[4])

[(('current next player bid 2 bit', 'next player turn order 1'), 2), (('current next player bid 2 bit', 'final player turn order 2'), 2), (('current next player bid 2 bit', 'final player turn order 1'), 2), (('next player turn order 1', 'final player turn order 2'), 2), (('next player turn order 1', 'final player turn order 1'), 2), (('final player turn order 2', 'final player turn order 1'), 2), (('final player coins bit 8', 'current final player bid 1'), 1), (('self next turn order 2', 'current next player bid 2 bit'), 1), (('self next turn order 2', 'next player turn order 1'), 1), (('self next turn order 2', 'final player turn order 2'), 1), (('self next turn order 2', 'final player turn order 1'), 1), (('resulting next player coins if fold 4', 'final player coins bit 8'), 1), (('resulting next player coins if fold 4', 'resulting final player coins if fold 8'), 1), (('final player coins bit 8', 'resulting final player coins if fold 8'), 1), (('current next player bid 8 bit', 'next 

In [103]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.01) ]

,support,itemsets,length


In [71]:
ensamble = [cuda_e1_tm, cuda_e2_tm, cuda_e3_tm]
y_test[20]

0

In [91]:
def ensamble_apriori(x, y, ensamble, sigma=-100000):
    print(y)
    ensamble_data = []
    for tm in ensamble:
        pre_ap = prepare_tm_for_apriori(x, y, tm, sigma)
        check_clause = [show_coalesced_clause_4r(tm, clause).strip().split("∧") for i, clause in enumerate(pre_ap) if show_coalesced_weight(tm, y, clause) > sigma]
        new_check = []
        for clause in check_clause:
            temp = []
            for feat in clause:
                temp.append(feat.strip())
            ensamble_data.append(temp)
    te = TransactionEncoder()
    te_ary = te.fit(ensamble_data).transform(ensamble_data)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    return df
df = ensamble_apriori(x_test[20], y_test[20], ensamble, sigma=0)
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
pd.set_option('max_colwidth', 1000)
frequent_itemsets

0


,support,itemsets,length
0,0.023833,(0 stack bit on),1
1,0.028798,(1 stack bit on),1
2,0.011917,(10 stack bit on),1
3,0.021847,(11 stack bit on),1
4,0.019861,(12 stack bit on),1
...,...,...,...
410,0.010924,"(¬resulting final player coins if fold 1, ¬own nr 4 card bit 8)",2
411,0.011917,"(¬resulting card on fold 2, ¬resulting card on fold 4)",2
412,0.010924,"(¬resulting card on fold 8, ¬resulting card on fold 4)",2
413,0.015889,"(¬resulting final player coins if fold 1, ¬resulting card on fold 4)",2


In [73]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.02) ]
# frequent_itemsets[ (frequent_itemsets['length'] == 2)]

,support,itemsets,length
144,0.020856,"(¬current next player bid 16 bit, board cards 1. bit 8)",2
204,0.022503,"(final player turn order 1, ¬own nr 4 card bit 2)",2
205,0.020856,"(final player turn order 1, ¬own nr 4 card bit 4)",2
261,0.021954,"(¬board cards 2. bit 1, ¬own nr 4 card bit 2)",2
281,0.021405,"(¬current next player bid 16 bit, ¬current final player bid 16)",2
322,0.024698,"(¬current next player bid 16 bit, ¬current own bid 16)",2
331,0.020856,"(¬current next player bid 16 bit, ¬next player nr 4 card bit 4)",2
352,0.020856,"(¬current own bid 16, ¬own nr 3 card bit 1)",2
449,0.020856,"(¬own nr 3 card bit 1, ¬next player nr 4 card bit 2)",2
453,0.025796,"(¬own nr 4 card bit 1, ¬next player nr 4 card bit 2)",2


In [106]:
y_test[5]

3

In [93]:
# clauses_out1.shape
mw = cuda_e1_tm.weight_banks[y_test[0]].get_weights()
weight_to_plot = mw[clauses_out1]
result = np.sort(weight_to_plot)
arg_result = np.argsort(weight_to_plot)

clauses_out1_r = np.where(weight_to_plot == weight_to_plot[abs(weight_to_plot) > 40])[1].astype(int)
# se distribution for 3, 11.4 og 19.8

/tmp/ipykernel_516615/2222734040.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  clauses_out1_r = np.where(weight_to_plot == weight_to_plot[abs(weight_to_plot) > 40])[1].astype(int)


IndexError: tuple index out of range

In [ ]:
ax = sns.histplot(weight_to_plot[abs(weight_to_plot) > 40])
clauses_out1_r = np.where(abs(weight_to_plot) > 40)[0]
text_output = [show_coalesced_clause_4r(cuda_e1_tm, clause) for clause in clauses_out1_r]
freqItemSet, rules = apriori(text_output, minSup=0.5, minConf=0.5)
rules


Clause #0:  Clause #3:  Clause #6:  Clause #8:  Clause #13:  Clause #15:  Clause #17:  Clause #18:  Clause #19:  Clause #20:  Clause #25:  Clause #26:  Clause #28:  Clause #30:  Clause #38:  Clause #40:  Clause #44:  Clause #46:  Clause #54:  Clause #58:  Clause #59:  Clause #62:  Clause #78:  Clause #82:  Clause #83:  Clause #88:  Clause #93:  Clause #96:  Clause #97:  Clause #101:  Clause #107:  Clause #115:  Clause #121:  Clause #123:  Clause #124:  Clause #125:  Clause #127:  Clause #128:  Clause #129:  Clause #131:  Clause #137:  Clause #138:  Clause #139:  Clause #141:  Clause #145:  Clause #147:  Clause #148:  Clause #156:  Clause #160:  Clause #164:  Clause #168:  Clause #169:  Clause #170:  Clause #175:  Clause #179:  Clause #187:  Clause #192:  Clause #195:  Clause #198:  Clause #199:  Clause #202:  Clause #205:  Clause #208:  Clause #213:  Clause #222:  Clause #223:  Clause #228:  Clause #229:  Clause #235:  Clause #237:  Clause #238:  Clause #240:  Clause #241:  Clause #242